# Main model classic double robust DiD

Note: 
- I caluzclate the average Variance not asymptotic Variance. i dont know what that is tbh

In [ ]:
import numpy as np
from doubleml import DoubleMLData, DoubleMLDID
from doubleml.datasets import make_did_SZ2020

check the new paper I found and this repository to replicate the results of sant'anna and Zhao (2021) paper. https://github.com/tommaso-manfe/DR-DIPW-Updated-Beyond-Regression-DiD/tree/main

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression

ml_g = LinearRegression()  # as in the paper, estimators not needed
ml_m = LogisticRegression()  # as in the paper, estimators not needed

In [ ]:
n_obs = 1000
n_rep = 1000
ATTE = 0.0

ATTE_estimates = np.full((n_rep), np.nan)
coverage = np.full((n_rep), np.nan)
ci_length = np.full((n_rep), np.nan)
asymptotic_variance = np.full(n_rep, np.nan)


np.random.seed(42)
for i_rep in range(n_rep):
    if (i_rep % int(n_rep / 10)) == 0:
        print(f"Iteration: {i_rep}/{n_rep}")
    dml_data = make_did_SZ2020(n_obs=n_obs, dgp_type=4, cross_sectional_data=False)

    dml_did = DoubleMLDID(dml_data, ml_g=ml_g, ml_m=ml_m, n_folds=5)
    dml_did.fit()

    ATTE_estimates[i_rep] = dml_did.coef.squeeze()
    confint = dml_did.confint(level=0.95)
    coverage[i_rep] = (confint["2.5 %"].iloc[0] <= ATTE) & (
        confint["97.5 %"].iloc[0] >= ATTE
    )
    ci_length[i_rep] = confint["97.5 %"].iloc[0] - confint["2.5 %"].iloc[0]

    summary_df = dml_did.summary
    std_err = summary_df.loc["d", "std err"]
    asymptotic_variance[i_rep] = std_err**2

# Calculate metrics
avg_bias = np.mean(ATTE_estimates - ATTE)
med_bias = np.median(ATTE_estimates - ATTE)
rmse = np.sqrt(np.mean((ATTE_estimates - ATTE) ** 2))
avg_asymptotic_variance = np.mean(asymptotic_variance)
coverage_probability = np.mean(coverage)
avg_ci_length = np.mean(ci_length)

# Print results
print(f"Av. Bias: {avg_bias}")
print(f"Med. Bias: {med_bias}")
print(f"RMSE: {rmse}")
print(f"Asy. V: {avg_asymptotic_variance}")
print(f"Cover: {coverage_probability}")
print(f"CIL: {avg_ci_length}")

# TWFE Simulation

DGP 1

In [ ]:
# DGP1

import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import logistic

np.random.seed(42)  # You can use any integer value as the seed

# Define parameters
n = 1000  # Sample size
Xsi_ps = 0.75  # pscore index
_lambda = 0.5  # Proportion in each period
nboot = 199  # Number of bootstrapped draws

# Define means and standard deviations
mean_z1 = np.exp(0.25 / 2)
sd_z1 = np.sqrt((np.exp(0.25) - 1) * np.exp(0.25))
mean_z2 = 10
sd_z2 = 0.54164
mean_z3 = 0.21887
sd_z3 = 0.04453
mean_z4 = 402
sd_z4 = 56.63891

# Initialize empty lists to store results
ATTE_estimates = []
asymptotic_variance = []
coverage = []
ci_length = []

# Loop for 1000 runs
for _i in range(1000):
    # Generate covariates
    x1 = np.random.normal(0, 1, n)
    x2 = np.random.normal(0, 1, n)
    x3 = np.random.normal(0, 1, n)
    x4 = np.random.normal(0, 1, n)

    z1 = np.exp(x1 / 2)
    z2 = x2 / (1 + np.exp(x1)) + 10
    z3 = (x1 * x3 / 25 + 0.6) ** 3
    z4 = (x1 + x4 + 20) ** 2

    z1 = (z1 - mean_z1) / sd_z1
    z2 = (z2 - mean_z2) / sd_z2
    z3 = (z3 - mean_z3) / sd_z3
    z4 = (z4 - mean_z4) / sd_z4

    x = np.column_stack((x1, x2, x3, x4))
    z = np.column_stack((z1, z2, z3, z4))

    # Propensity score
    pi = logistic.cdf(Xsi_ps * (-z1 + 0.5 * z2 - 0.25 * z3 - 0.1 * z4))
    d = np.random.uniform(size=n) <= pi

    # Generate aux indexes for the potential outcomes
    index_lin = 210 + 27.4 * z1 + 13.7 * (z2 + z3 + z4)
    index_unobs_het = d * (index_lin)
    index_att = 0
    index_trend = 210 + 27.4 * z1 + 13.7 * (z2 + z3 + z4)

    # Generate unobserved heterogeneity
    v = np.random.normal(index_unobs_het, 1)

    # Generate outcomes at time 0 and time 1
    y00 = index_lin + v + np.random.normal(size=n)
    y10 = index_lin + v + np.random.normal(size=n)
    y01 = index_lin + v + np.random.normal(scale=1, size=n) + index_trend
    y11 = index_lin + v + np.random.normal(scale=1, size=n) + index_trend + index_att

    # Generate "T"
    ti_nt = 0.5
    ti_t = 0.5
    ti = d * ti_t + (1 - d) * ti_nt
    post = np.random.uniform(size=n) <= ti

    y = (
        d * post * y11
        + (1 - d) * post * y01
        + (1 - d) * (1 - post) * y00
        + (d) * (1 - post) * y10
    )

    # Generate id
    id_ = np.arange(1, n + 1)

    # Put in a long data frame
    dta_long = pd.DataFrame(
        {
            "id": id_,
            "y": y,
            "post": post.astype(int),
            "d": d.astype(int),
            "x1": z1,
            "x2": z2,
            "x3": z3,
            "x4": z4,
        },
    )
    dta_long["post:d"] = dta_long["post"] * dta_long["d"]

    dta_long = dta_long.sort_values(by="id")

    # Perform TWFE estimation
    twfe_i = sm.OLS(
        dta_long["y"],
        sm.add_constant(dta_long[["x1", "x2", "x3", "x4", "post", "d", "post:d"]]),
    ).fit()
    twfe = twfe_i.params["post:d"]

    # Calculate asymptotic variance
    asymptotic_variance.append(twfe_i.cov_params()["post:d"])

    # Bootstrap for confidence intervals
    # (Note: This part is not included in the provided code, you'll need to implement it separately)

    # Append TWFE estimate to the list
    ATTE_estimates.append(twfe)

# Convert lists to arrays for ease of calculation
ATTE_estimates = np.array(ATTE_estimates)
asymptotic_variance = np.array(asymptotic_variance)
asymptotic_variance = asymptotic_variance.reshape(-1, 8)
asymptotic_variance = asymptotic_variance[:, 0]

# Calculate metrics
avg_bias = np.mean(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
med_bias = np.median(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
rmse = np.sqrt(
    np.mean((ATTE_estimates - 0) ** 2),
)  # Assuming ATTE is 0 as mentioned in the code
avg_asymptotic_variance = np.mean(asymptotic_variance)
coverage_probability = np.mean(
    ((ATTE_estimates - 1.96 * np.sqrt(asymptotic_variance)) <= 0)
    & ((ATTE_estimates + 1.96 * np.sqrt(asymptotic_variance)) >= 0),
)
avg_ci_length = np.mean(
    2 * 1.96 * np.sqrt(asymptotic_variance),
)  # Length of 95% confidence interval
average_ATT_estimate = np.mean(ATTE_estimates)

# Print results
print(f"Average ATT estimate across all runs: {average_ATT_estimate}")

print(f"Av. Bias: {avg_bias}")
print(f"Med. Bias: {med_bias}")
print(f"RMSE: {rmse}")
print(f"Asy. V: {avg_asymptotic_variance}")
print(f"Cover: {coverage_probability}")
print(f"CIL: {avg_ci_length}")

DGP 2

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import logistic

np.random.seed(42)  # You can use any integer value as the seed

# Sample size
n = 1000
# pscore index (strength of common support)
Xsi_ps = 0.75
# Proportion in each period
_lambda = 0.5
# Number of bootstrapped draws
nboot = 199

# Mean and Std deviation of Z's without truncation
mean_z1 = np.exp(0.25 / 2)
sd_z1 = np.sqrt((np.exp(0.25) - 1) * np.exp(0.25))
mean_z2 = 10
sd_z2 = 0.54164
mean_z3 = 0.21887
sd_z3 = 0.04453
mean_z4 = 402
sd_z4 = 56.63891


# Initialize empty lists to store results
ATTE_estimates = []
asymptotic_variance = []
coverage = []
ci_length = []


for _i in range(1000):
    # Generate covariates
    x1 = np.random.normal(0, 1, n)
    x2 = np.random.normal(0, 1, n)
    x3 = np.random.normal(0, 1, n)
    x4 = np.random.normal(0, 1, n)

    z1 = np.exp(x1 / 2)
    z2 = x2 / (1 + np.exp(x1)) + 10
    z3 = (x1 * x3 / 25 + 0.6) ** 3
    z4 = (x1 + x4 + 20) ** 2

    z1 = (z1 - mean_z1) / sd_z1
    z2 = (z2 - mean_z2) / sd_z2
    z3 = (z3 - mean_z3) / sd_z3
    z4 = (z4 - mean_z4) / sd_z4

    x = np.column_stack((x1, x2, x3, x4))
    z = np.column_stack((z1, z2, z3, z4))

    # Generate treatment groups
    # Propensity score
    pi = logistic.cdf(Xsi_ps * (-x1 + 0.5 * x2 - 0.25 * x3 - 0.1 * x4))
    d = np.random.uniform(size=n) <= pi

    # Generate aux indexes for the potential outcomes
    index_lin = 210 + 27.4 * z1 + 13.7 * (z2 + z3 + z4)

    # Create heterogenenous effects for the ATT, which is set approximately equal to zero
    index_unobs_het = d * (index_lin)
    index_att = 0

    # This is the key for consistency of outcome regression
    index_trend = 210 + 27.4 * z1 + 13.7 * (z2 + z3 + z4)

    # v is the unobserved heterogeneity
    v = np.random.normal(index_unobs_het, 1)

    # Gen realized outcome at time 0
    y00 = index_lin + v + np.random.normal(size=n)
    y10 = index_lin + v + np.random.normal(size=n)

    # Gen outcomes at time 1
    # First let's generate potential outcomes: y_1_potential
    y01 = (
        index_lin + v + np.random.normal(scale=1, size=n) + index_trend
    )  # This is the baseline
    y11 = (
        index_lin + v + np.random.normal(scale=1, size=n) + index_trend + index_att
    )  # This is the baseline

    # Generate "T"
    ti_nt = 0.5
    ti_t = 0.5
    ti = d * ti_t + (1 - d) * ti_nt
    post = np.random.uniform(size=n) <= ti

    y = (
        d * post * y11
        + (1 - d) * post * y01
        + (1 - d) * (1 - post) * y00
        + (d) * (1 - post) * y10
    )

    # Gen id
    id_ = np.arange(1, n + 1)

    # Put in a long data frame
    dta_long = pd.DataFrame(
        {
            "id": id_,
            "y": y,
            "post": post.astype(int),
            "d": d.astype(int),
            "x1": z1,
            "x2": z2,
            "x3": z3,
            "x4": z4,
        },
    )
    dta_long = dta_long.sort_values(by="id")

    # Create interaction term 'post:d'
    dta_long["post:d"] = dta_long["post"] * dta_long["d"]

    twfe_i = sm.OLS(
        dta_long["y"],
        sm.add_constant(dta_long[["x1", "x2", "x3", "x4", "post", "d", "post:d"]]),
    ).fit()
    twfe = twfe_i.params["post:d"]

    asymptotic_variance.append(
        twfe_i.HC0_se["post:d"],
    )  # Append the standard error for the interaction term


# Convert lists to arrays for calculations
twfe = np.array(twfe)
asymptotic_variance = np.array(asymptotic_variance)

# Calculate metrics
avg_bias = np.mean(twfe - 0)  # Assuming ATTE is 0 as mentioned in the code
med_bias = np.median(twfe - 0)  # Assuming ATTE is 0 as mentioned in the code
rmse = np.sqrt(np.mean((twfe - 0) ** 2))  # Assuming ATTE is 0 as mentioned in the code
avg_asymptotic_variance = np.mean(asymptotic_variance)
coverage_probability = np.mean(
    ((twfe - 1.96 * np.sqrt(asymptotic_variance)) <= 0)
    & ((twfe + 1.96 * np.sqrt(asymptotic_variance)) >= 0),
)
avg_ci_length = np.mean(
    2 * 1.96 * np.sqrt(asymptotic_variance),
)  # Length of 95% confidence interval

# Print results
print(f"Av. Bias: {avg_bias}")
print(f"Med. Bias: {med_bias}")
print(f"RMSE: {rmse}")
print(f"Asy. V: {avg_asymptotic_variance}")
print(f"Cover: {coverage_probability}")
print(f"CIL: {avg_ci_length}")

DGP 3

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

np.random.seed(42)  # You can use any integer value as the seed

# Sample size
n = 1000
# pscore index (strength of common support)
Xsi_ps = 0.75
# Proportion in each period
_lambda = 0.5
nboot = 199  # Number of bootstrapped draws


# Mean and Std deviation of Z's without truncation
mean_z1 = np.exp(0.25 / 2)
sd_z1 = np.sqrt((np.exp(0.25) - 1) * np.exp(0.25))
mean_z2 = 10
sd_z2 = 0.54164
mean_z3 = 0.21887
sd_z3 = 0.04453
mean_z4 = 402
sd_z4 = 56.63891


# Initialize empty lists to store results
ATTE_estimates = []
asymptotic_variance = []
coverage = []
ci_length = []

for _i in range(1000):
    # Generate covariates
    x1 = np.random.normal(0, 1, n)
    x2 = np.random.normal(0, 1, n)
    x3 = np.random.normal(0, 1, n)
    x4 = np.random.normal(0, 1, n)

    z1 = np.exp(x1 / 2)
    z2 = x2 / (1 + np.exp(x1)) + 10
    z3 = (x1 * x3 / 25 + 0.6) ** 3
    z4 = (x1 + x4 + 20) ** 2

    z1 = (z1 - mean_z1) / sd_z1
    z2 = (z2 - mean_z2) / sd_z2
    z3 = (z3 - mean_z3) / sd_z3
    z4 = (z4 - mean_z4) / sd_z4

    x = np.column_stack((x1, x2, x3, x4))
    z = np.column_stack((z1, z2, z3, z4))

    # Propensity score
    pi = 1 / (1 + np.exp(Xsi_ps * (-z1 + 0.5 * z2 - 0.25 * z3 - 0.1 * z4)))
    d = np.random.uniform(size=n) <= pi

    # Generate aux indexes for the potential outcomes
    index_lin = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)

    # Create heterogenenous effects for the ATT, which is set approximately equal to zero
    index_unobs_het = d * index_lin
    index_att = 0

    # This is the key for consistency of outcome regression
    index_trend = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)
    # v is the unobserved heterogeneity
    v = np.random.normal(index_unobs_het, 1)

    # Gen realized outcome at time 0
    y00 = index_lin + v + np.random.normal(size=n)
    y10 = index_lin + v + np.random.normal(size=n)

    # Gen outcomes at time 1
    y01 = index_lin + v + np.random.normal(size=n) + index_trend  # This is the baseline
    y11 = (
        index_lin + v + np.random.normal(size=n) + index_trend + index_att
    )  # This is the baseline

    # Generate "T"
    ti_nt = 0.5
    ti_t = 0.5
    ti = d * ti_t + (1 - d) * ti_nt
    post = np.random.uniform(size=n) <= ti

    y = (
        d * post * y11
        + (1 - d) * post * y01
        + (1 - d) * (1 - post) * y00
        + (d) * (1 - post) * y10
    )

    # Gen id
    id_ = np.arange(1, n + 1)

    # Put in a long data frame
    dta_long = pd.DataFrame(
        {
            "id": id_,
            "y": y,
            "post": post.astype(int),
            "d": d.astype(int),
            "x1": z1,
            "x2": z2,
            "x3": z3,
            "x4": z4,
        },
    )
    dta_long = dta_long.sort_values(by="id")
    # Create interaction term 'post:d'
    dta_long["post:d"] = dta_long["post"] * dta_long["d"]
    # Standard TWFE

    twfe_i = sm.OLS(
        dta_long["y"],
        sm.add_constant(dta_long[["x1", "x2", "x3", "x4", "post", "d", "post:d"]]),
    ).fit()
    twfe = twfe_i.params["post:d"]
    asymptotic_variance.append(twfe_i.cov_params()["post:d"])
    # Convert lists to arrays for calculations
    ATTE_estimates.append(twfe)


# Convert lists to arrays for ease of calculation
ATTE_estimates = np.array(ATTE_estimates)
asymptotic_variance = np.array(asymptotic_variance)
asymptotic_variance = asymptotic_variance.reshape(-1, 8)
asymptotic_variance = asymptotic_variance[:, 0]

# Calculate metrics
avg_bias = np.mean(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
med_bias = np.median(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
rmse = np.sqrt(
    np.mean((ATTE_estimates - 0) ** 2),
)  # Assuming ATTE is 0 as mentioned in the code
avg_asymptotic_variance = np.mean(asymptotic_variance)
coverage_probability = np.mean(
    ((ATTE_estimates - 1.96 * np.sqrt(asymptotic_variance)) <= 0)
    & ((ATTE_estimates + 1.96 * np.sqrt(asymptotic_variance)) >= 0),
)
avg_ci_length = np.mean(
    2 * 1.96 * np.sqrt(asymptotic_variance),
)  # Length of 95% confidence interval
average_ATT_estimate = np.mean(ATTE_estimates)

# Print results
print(f"Average ATT estimate across all runs: {average_ATT_estimate}")

print(f"Av. Bias: {avg_bias}")
print(f"Med. Bias: {med_bias}")
print(f"RMSE: {rmse}")
print(f"Asy. V: {avg_asymptotic_variance}")
print(f"Cover: {coverage_probability}")
print(f"CIL: {avg_ci_length}")

DGP 4

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

np.random.seed(42)  # You can use any integer value as the seed
# Sample size
n = 1000

# pscore index (strength of common support)
Xsi_ps = 0.75

# Proportion in each period
lambda_ = 0.5

# Number of bootstrapped draws
nboot = 199

# Mean and Std deviation of Z's without truncation
mean_z1 = np.exp(0.25 / 2)
sd_z1 = np.sqrt((np.exp(0.25) - 1) * np.exp(0.25))
mean_z2 = 10
sd_z2 = 0.54164
mean_z3 = 0.21887
sd_z3 = 0.04453
mean_z4 = 402
sd_z4 = 56.63891


# Initialize empty lists to store results
ATTE_estimates = []
asymptotic_variance = []
coverage = []
ci_length = []


for _i in range(1000):
    # Gen covariates
    x1 = np.random.normal(0, 1, n)
    x2 = np.random.normal(0, 1, n)
    x3 = np.random.normal(0, 1, n)
    x4 = np.random.normal(0, 1, n)

    z1 = np.exp(x1 / 2)
    z2 = x2 / (1 + np.exp(x1)) + 10
    z3 = (x1 * x3 / 25 + 0.6) ** 3
    z4 = (x1 + x4 + 20) ** 2

    z1 = (z1 - mean_z1) / sd_z1
    z2 = (z2 - mean_z2) / sd_z2
    z3 = (z3 - mean_z3) / sd_z3
    z4 = (z4 - mean_z4) / sd_z4

    x = np.column_stack((x1, x2, x3, x4))
    z = np.column_stack((z1, z2, z3, z4))

    # Gen treatment groups
    # Propensity score
    pi = 1 / (1 + np.exp(-Xsi_ps * (-x1 + 0.5 * x2 - 0.25 * x3 - 0.1 * x4)))
    d = np.random.rand(n) <= pi

    # Generate aux indexes for the potential outcomes
    index_lin = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)

    # Create heterogeneous effects for the ATT, which is set approximately equal to zero
    index_unobs_het = d * index_lin
    index_att = 0

    # This is the key for consistency of outcome regression
    index_trend = 210 + 27.4 * x1 + 13.7 * (x2 + x3 + x4)

    # v is the unobserved heterogeneity
    v = np.random.normal(index_unobs_het, 1, n)

    # Gen realized outcome at time 0
    y00 = index_lin + v + np.random.normal(size=n)
    y10 = index_lin + v + np.random.normal(size=n)

    # Gen outcomes at time 1
    # First let's generate potential outcomes: y_1_potential
    y01 = index_lin + v + np.random.normal(size=n) + index_trend
    y11 = index_lin + v + np.random.normal(size=n) + index_trend + index_att

    # Generate "T"
    ti_nt = 0.5
    ti_t = 0.5
    ti = d * ti_t + (1 - d) * ti_nt
    post = np.random.rand(n) <= ti

    y = (
        d * post * y11
        + (1 - d) * post * y01
        + (1 - d) * (1 - post) * y00
        + d * (1 - post) * y10
    )

    # Gen id
    id_ = np.arange(1, n + 1)

    # Put in a long data frame
    dta_long = pd.DataFrame(
        {
            "id": id_,
            "y": y,
            "post": post.astype(int),
            "d": d.astype(int),
            "x1": z1,
            "x2": z2,
            "x3": z3,
            "x4": z4,
        },
    )

    dta_long = dta_long.sort_values("id")
    dta_long["post:d"] = dta_long["post"] * dta_long["d"]
    # Standard TWFE
    twfe_i = sm.OLS(
        dta_long["y"],
        sm.add_constant(dta_long[["x1", "x2", "x3", "x4", "post", "d", "post:d"]]),
    ).fit()
    twfe = twfe_i.params["post:d"]
    asymptotic_variance.append(twfe_i.cov_params()["post:d"])
    ATTE_estimates.append(twfe)
# Convert lists to arrays for calculations

# Calculate metrics

# Convert lists to arrays for ease of calculation
ATTE_estimates = np.array(ATTE_estimates)
asymptotic_variance = np.array(asymptotic_variance)
asymptotic_variance = asymptotic_variance.reshape(-1, 8)
asymptotic_variance = asymptotic_variance[:, 0]

# Calculate metrics
avg_bias = np.mean(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
med_bias = np.median(ATTE_estimates - 0)  # Assuming ATTE is 0 as mentioned in the code
rmse = np.sqrt(
    np.mean((ATTE_estimates - 0) ** 2),
)  # Assuming ATTE is 0 as mentioned in the code
avg_asymptotic_variance = np.mean(asymptotic_variance)
coverage_probability = np.mean(
    ((ATTE_estimates - 1.96 * np.sqrt(asymptotic_variance)) <= 0)
    & ((ATTE_estimates + 1.96 * np.sqrt(asymptotic_variance)) >= 0),
)
avg_ci_length = np.mean(
    2 * 1.96 * np.sqrt(asymptotic_variance),
)  # Length of 95% confidence interval
average_ATT_estimate = np.mean(ATTE_estimates)

# Print results
print(f"Average ATT estimate across all runs: {average_ATT_estimate}")

print(f"Av. Bias: {avg_bias}")
print(f"Med. Bias: {med_bias}")
print(f"RMSE: {rmse}")
print(f"Asy. V: {avg_asymptotic_variance}")
print(f"Cover: {coverage_probability}")
print(f"CIL: {avg_ci_length}")

# Deep learning with a few runs

be careful overfitting could be a problem here!!

In [ ]:
import warnings

import numpy as np
import tensorflow as tf
from doubleml import DoubleMLData, DoubleMLDID
from doubleml.datasets import make_did_SZ2020
from scikeras.wrappers import KerasClassifier  # pip install scikeras
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

warnings.filterwarnings("ignore")
np.random.seed(42)
n_reps = 10  # change that accordingly
n_obs = 1000
ATTE = 0.0

# Enable GPU acceleration if available
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

# Function to create Keras model


def create_model():
    model = Sequential()
    model.add(Dense(32, input_dim=x.shape[1], activation="relu"))  # Simplified
    model.add(Dense(16, activation="relu"))  # Simplified
    model.add(Dense(1, activation="sigmoid"))  # Assuming binary classification
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model


# Initialize arrays to store statistics
biases = np.full(n_reps, np.nan)
variances = np.full(n_reps, np.nan)
rmse_list = np.full(n_reps, np.nan)
coverage = np.full(n_reps, np.nan)
asymptotic_variance = np.full(n_reps, np.nan)
ci_length = np.full(n_reps, np.nan)
ATTE_estimates = np.full(n_reps, np.nan)

early_stopping = EarlyStopping(monitor="loss", patience=2, verbose=0)  # Early stopping

for i_rep in range(n_reps):
    x, y, d = make_did_SZ2020(
        n_obs=n_obs,
        dgp_type=4,
        cross_sectional_data=False,
        return_type="array",
    )
    dml_data = DoubleMLData.from_arrays(x=x, y=y, d=d)

    # Wrap the Keras model with KerasClassifier
    keras_classifier = KerasClassifier(
        build_fn=create_model,
        epochs=3,  # adjust as needed
        batch_size=32,  # Smaller batch size
        verbose=0,
        callbacks=[early_stopping],
    )

    # Use StandardScaler to normalize data and then use the Keras classifier in a pipeline
    ml_m = Pipeline([("scaler", StandardScaler()), ("nn", keras_classifier)])

    # Use LinearRegression for regression
    ml_g = LinearRegression()

    dml_plr = DoubleMLDID(dml_data, ml_g, ml_m)
    dml_plr.fit()

    ATTE_estimates[i_rep] = dml_plr.coef.squeeze()
    confint = dml_plr.confint(level=0.95)
    coverage[i_rep] = (confint["2.5 %"].iloc[0] <= ATTE) & (
        confint["97.5 %"].iloc[0] >= ATTE
    )
    ci_length[i_rep] = confint["97.5 %"].iloc[0] - confint["2.5 %"].iloc[0]
    # Extract standard error from the summary
    summary_df = dml_plr.summary
    std_err = summary_df.loc["d", "std err"]
    asymptotic_variance[i_rep] = std_err**2

# Calculate metrics
avg_bias = np.mean(ATTE_estimates - ATTE)
med_bias = np.median(ATTE_estimates - ATTE)
rmse = np.sqrt(np.mean((ATTE_estimates - ATTE) ** 2))
avg_asymptotic_variance = np.mean(asymptotic_variance)
coverage_probability = np.mean(coverage)
avg_ci_length = np.mean(ci_length)

# Print results
print(f"Av. Bias: {avg_bias}")
print(f"Med. Bias: {med_bias}")
print(f"RMSE: {rmse}")
print(f"Asy. V: {avg_asymptotic_variance}")
print(f"Cover: {coverage_probability}")
print(f"CIL: {avg_ci_length}")

## Visulaization Ideas

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

df_pa = pd.DataFrame(ATTE_estimates, columns=["Estimate"])
g = sns.kdeplot(df_pa, fill=True)
plt.show()

# Deep Learning without double robust

check the paper of the phd guys how they did the IPW or essentially the propensity score matching